In [1]:
!nvidia-smi

Fri May 13 16:48:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8     8W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
from datasets import load_dataset,Dataset,get_dataset_config_names,DatasetDict
import warnings # to ignore warnings
warnings.filterwarnings('ignore')

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [5]:
# training data
language = 'vi'
input_file_path = f'../../SQuAD/translate-train/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

In [6]:
train.head()

,index,question,context,answer_start,text,c_id
0,5726e1755951b619008f8141,Tên của nhà vật lý người Hà Lan đã phát minh r...,"Vào tháng 10 năm 1745, Ewald Georg von Kleist ...",549,Pieter van Musschenbroek,0
1,572fde9204bcaa1900d76e05,Môn thể thao Olympic nào mà Hrant Shahinyan th...,"Trước năm 1992, người Armenia sẽ tham gia Thế ...",430,thể dục dụng cụ,1
2,56d37d7f59d6e4140014651a,Vòng loại nào Chris Medina đã bị loại trong mù...,Một trong những thí sinh nổi bật hơn trong năm...,191,Top 40,2
3,57318dc8a5e9cc1400cdc061,Spielberg đã ký bao nhiêu bộ phim truyền hình?,"Dựa trên sức mạnh công việc của mình, Universa...",77,bốn,3
4,57331b0dd058e614000b56fd,Lý thuyết chủ yếu của thực tế mà Whitehead phả...,Bắt đầu từ cuối những năm 1910 và đầu những nă...,469,thực tế được xây dựng cơ bản bởi các bit vật c...,4


In [7]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5726e1755951b619008f8141,Tên của nhà vật lý người Hà Lan đã phát minh ra Leyden Jar là gì?,"Vào tháng 10 năm 1745, Ewald Georg von Kleist ở Pomerania, Đức, nhận thấy rằng điện tích có thể được lưu trữ bằng cách kết nối một máy phát tĩnh điện cao áp bằng một sợi dây với một thể tích nước trong một lọ thủy tinh cầm tay. Tay và nước của Von Kleist đóng vai trò là chất dẫn điện và bình như một chất điện môi (mặc dù chi tiết về cơ chế được xác định không chính xác tại thời điểm đó). Von Kleist nhận thấy rằng việc chạm vào dây dẫn đến một tia lửa mạnh mẽ, đau đớn hơn nhiều so với việc lấy từ máy tĩnh điện. Năm sau, nhà vật lý người Hà Lan Pieter van Musschenbroek đã phát minh ra một tụ điện tương tự, được đặt tên là bình Leyden, sau Đại học Leiden nơi ông làm việc. Ông cũng bị ấn tượng bởi sức mạnh của cú sốc mà ông nhận được, viết: ""Tôi sẽ không chịu cú sốc thứ hai cho vương quốc Pháp.""",549,Pieter van Musschenbroek,0,"{'answer_start': [549], 'text': ['Pieter van Musschenbroek']}"


In [9]:
# Validation data
language = 'vi'
input_file_path = f'../../SQuAD/translate-dev/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

,index,question,context,answer_start,text,c_id,answers
0,5726e179dd62a815002e93b1,Những dược sĩ nào có khả năng tìm kiếm giáo dục bổ sung sau trường dược?,"Do sự phức tạp của thuốc bao gồm chỉ định cụ thể, hiệu quả của chế độ điều trị, an toàn của thuốc (nghĩa là tương tác thuốc) và các vấn đề tuân thủ của bệnh nhân (trong bệnh viện và tại nhà), nhiều dược sĩ thực hành tại bệnh viện được giáo dục và đào tạo nhiều hơn sau khi học trường dược một cư dân hành nghề dược và đôi khi theo sau là một cư dân khác trong một khu vực cụ thể. Những dược sĩ đó thường được gọi là dược sĩ lâm sàng và họ thường chuyên về các ngành dược khác nhau. Ví dụ, có những dược sĩ chuyên về huyết học / ung thư, HIV / AIDS, bệnh truyền nhiễm, chăm sóc quan trọng, thuốc cấp cứu, độc dược, dược học hạt nhân, quản lý đau, tâm thần, phòng chống đông máu, thảo dược, quản lý thần kinh / động kinh, nhi khoa , dược sĩ sơ sinh và nhiều hơn nữa.",416,dược sĩ lâm sàng,0,"{'answer_start': [416], 'text': ['dược sĩ lâm sàng']}"


In [10]:
dev.head()

,index,question,context,answer_start,text,c_id,answers
0,5726e179dd62a815002e93b1,Những dược sĩ nào có khả năng tìm kiếm giáo dục bổ sung sau trường dược?,"Do sự phức tạp của thuốc bao gồm chỉ định cụ thể, hiệu quả của chế độ điều trị, an toàn của thuốc (nghĩa là tương tác thuốc) và các vấn đề tuân thủ của bệnh nhân (trong bệnh viện và tại nhà), nhiều dược sĩ thực hành tại bệnh viện được giáo dục và đào tạo nhiều hơn sau khi học trường dược một cư dân hành nghề dược và đôi khi theo sau là một cư dân khác trong một khu vực cụ thể. Những dược sĩ đó thường được gọi là dược sĩ lâm sàng và họ thường chuyên về các ngành dược khác nhau. Ví dụ, có những dược sĩ chuyên về huyết học / ung thư, HIV / AIDS, bệnh truyền nhiễm, chăm sóc quan trọng, thuốc cấp cứu, độc dược, dược học hạt nhân, quản lý đau, tâm thần, phòng chống đông máu, thảo dược, quản lý thần kinh / động kinh, nhi khoa , dược sĩ sơ sinh và nhiều hơn nữa.",416,dược sĩ lâm sàng,0,"{'answer_start': [416], 'text': ['dược sĩ lâm sàng']}"
1,5727580bf1498d1400e8f69e,Cách đánh vần nào của Genghis phù hợp nhất với cách phát âm có thể xảy ra của nó?,"Một giả thuyết cho rằng cái tên này bắt nguồn từ một phiên bản được khai hóa của từ tiếng Mông Cổ và tiếng Thổ Nhĩ Kỳ, có nghĩa là ""đại dương"", ""đại dương"" hoặc ""trải rộng"". (Hồ Baikal và đại dương được người Mông Cổ gọi là tenggis. Tuy nhiên, dường như nếu họ có ý định gọi Genghis tenggis thì họ có thể đã nói, và viết, ""Tenggis Khan"", mà họ không biết.) Zhèng (tiếng Trung:) có nghĩa là ""đúng"", ""chỉ"" hoặc ""đúng"", sẽ nhận được từ bổ nghĩa tính từ Mông Cổ -s, tạo ra ""Jenggis"", trong tiếng La Mã thời trung cổ sẽ được viết là ""Genghis"". Có khả năng cách phát âm tiếng Mông Cổ của thế kỷ 13 sẽ rất phù hợp với ""Chính tả"".",469,"""Jenggis",1,"{'answer_start': [469], 'text': ['""Jenggis']}"
2,572665ff708984140094c4c3,Nguyên tắc nào liên quan đến sự hình thành các lỗi và tuổi của các chuỗi mà chúng cắt?,"Nguyên tắc mối quan hệ xuyên suốt liên quan đến sự hình thành các lỗi và tuổi của các chuỗi mà qua đó chúng cắt. Lỗi là trẻ hơn những tảng đá họ cắt; theo đó, nếu một lỗi được tìm thấy thâm nhập vào một số hình dạng chứ không phải các lỗi trên nó, thì các hình dạng bị cắt cũ hơn lỗi và các phần không bị cắt phải nhỏ hơn lỗi. Tìm giường chính trong những tình huống này có thể giúp xác định xem lỗi là lỗi thông thường hay lỗi lực đẩy.",11,mối quan hệ xuyên suốt,2,"{'answer_start': [11], 'text': ['mối quan hệ xuyên suốt']}"
3,57308f6b8ab72b1400f9c581,Các văn bản phương Tây đầu tiên đề cập đến phương Đông mô tả người dân là gì?,"Chủ nghĩa phương Đông, theo lý thuyết của Edward Said, đề cập đến cách phương Tây phát triển một địa lý tưởng tượng của phương Đông. Địa lý tưởng tượng này dựa trên một diễn ngôn thiết yếu đại diện cho cả sự đa dạng cũng như thực tế xã hội của phương Đông. Thay vào đó, bằng cách thiết yếu phương Đông, bài diễn văn này sử dụng ý tưởng về bản sắc dựa trên địa điểm để tạo ra sự khác biệt và khoảng cách giữa ""chúng ta"" phương Tây và ""họ"" phương Đông, hay ""ở đây"" ở phương Tây và ""ở đó"" ở phương Đông. Sự khác biệt này đặc biệt rõ ràng trong các tác phẩm văn bản và hình ảnh của các nghiên cứu đầu tiên của Châu Âu về Phương Đông định vị phương Đông là phi lý và lạc hậu đối lập với phương Tây hợp lý và tiến bộ. Xác định phương Đông như một tầm nhìn tiêu cực của chính nó, vì nó kém hơn, không chỉ làm tăng ý thức về bản thân của West West, mà còn là một cách ra lệnh cho phương Đông và khiến phương Tây biết đến nó để có thể bị chi phối và kiểm soát. Do đó, diễn ngôn của chủ nghĩa phương Đông đóng vai trò là sự biện minh về ý thức hệ của chủ nghĩa đế quốc phương Tây thời kỳ đầu, vì nó hình thành một cơ thể tri thức và ý tưởng hợp lý hóa sự kiểm soát xã hội, văn hóa, chính trị và kinh tế của các lãnh thổ khác.",652,phi lý và lạc hậu,3,"{'answer_start': [652], 'text': ['phi lý và lạc hậu']}"
4,57308f6b8ab72b1400f9c581,Các văn bản phương Tây đầu tiên đề cập đến phương Đông mô tả người dân là gì?,"Chủ nghĩa phương Đ

In [11]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad = DatasetDict()

squad['train'] = tds
squad['validation'] = vds

In [12]:
squad

DatasetDict({
    train: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 87187
    })
    validation: Dataset({
        features: ['index', 'question', 'context', 'answer_start', 'text', 'c_id', 'answers'],
        num_rows: 34575
    })
})

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

pretraining_language = language
# model = AutoModelForQuestionAnswering.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")
model = AutoModelForQuestionAnswering.from_pretrained(f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

Downloading: 100%|██████████| 686/686 [00:00<00:00, 687kB/s]
Downloading: 100%|██████████| 450M/450M [00:27<00:00, 17.4MB/s]   
Some weights of the model checkpoint at subhasisj/vi-TAPT-MLM-MiniLM were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

In [14]:
pad_on_right = tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["index"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [15]:
batch_size = 8

In [16]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

100%|██████████| 35/35 [00:18<00:00,  1.89ba/s]


In [17]:
# squad["validation"] = squad["validation"].map(prepare_validation_features,batched=True,remove_columns=squad["validation"].column_names)

In [18]:
from transformers import Trainer, TrainingArguments
args = TrainingArguments(
    f"./{pretraining_language}-finetuned-squad-qa-minilmv2-{batch_size}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 3e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    warmup_ratio = 0.1,
    gradient_accumulation_steps = 8,
    num_train_epochs = 5,
    fp16=True,
    weight_decay = 0.01,
    push_to_hub=True,
    save_total_limit=1,
    load_best_model_at_end=True
)

In [19]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=squad["train"],
    eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/subhasisj/vi-finetuned-squad-qa-minilmv2-8 into local empty directory.
Using amp half precision backend


In [21]:
trainer.train()

***** Running training *****
  Num examples = 91135
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 7120
  7%|▋         | 500/7120 [11:10<2:22:36,  1.29s/it]

{'loss': 3.9579, 'learning_rate': 2.106741573033708e-05, 'epoch': 0.35}


 14%|█▍        | 1000/7120 [21:48<2:09:24,  1.27s/it]

{'loss': 2.1669, 'learning_rate': 2.8661048689138576e-05, 'epoch': 0.7}


 20%|██        | 1424/7120 [30:45<2:00:40,  1.27s/it]***** Running Evaluation *****
  Num examples = 36529
  Batch size = 8
                                                     
 20%|██        | 1424/7120 [34:24<2:00:40,  1.27s/it]Saving model checkpoint to ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-1424
Configuration saved in ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-1424\config.json


...

100%|██████████| 7120/7120 [2:52:11<00:00,  1.31s/it]***** Running Evaluation *****
  Num examples = 36529
  Batch size = 8
                                                     
100%|██████████| 7120/7120 [2:55:54<00:00,  1.31s/it]Saving model checkpoint to ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-7120
Configuration saved in ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-7120\config.json


{'eval_loss': 1.3335455656051636, 'eval_runtime': 222.8103, 'eval_samples_per_second': 163.947, 'eval_steps_per_second': 20.497, 'epoch': 5.0}


Model weights saved in ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-7120\pytorch_model.bin
tokenizer config file saved in ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-7120\tokenizer_config.json
Special tokens file saved in ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-7120\special_tokens_map.json
Deleting older checkpoint [vi-finetuned-squad-qa-minilmv2-8\checkpoint-4272] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./vi-finetuned-squad-qa-minilmv2-8\checkpoint-5696 (score: 1.3103300333023071).
100%|██████████| 7120/7120 [2:56:17<00:00,  1.49s/it]

{'train_runtime': 10577.9662, 'train_samples_per_second': 43.078, 'train_steps_per_second': 0.673, 'train_loss': 1.3618155436569386, 'epoch': 5.0}


TrainOutput(global_step=7120, training_loss=1.3618155436569386, metrics={'train_runtime': 10577.9662, 'train_samples_per_second': 43.078, 'train_steps_per_second': 0.673, 'train_loss': 1.3618155436569386, 'epoch': 5.0})

In [22]:
trainer.push_to_hub("Training Completed")

Saving model checkpoint to ./vi-finetuned-squad-qa-minilmv2-8
Configuration saved in ./vi-finetuned-squad-qa-minilmv2-8\config.json
Model weights saved in ./vi-finetuned-squad-qa-minilmv2-8\pytorch_model.bin
tokenizer config file saved in ./vi-finetuned-squad-qa-minilmv2-8\tokenizer_config.json
Special tokens file saved in ./vi-finetuned-squad-qa-minilmv2-8\special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
Upload file pytorch_model.bin: 100%|█████████▉| 448M/448M [1:34:10<00:00, 103kB/s]  remote: Enforcing permissions...        
remote: Allowed refs: all        
remote: error: cannot lock ref 'refs/heads/main': is at c5355ca2a0f4fe545c53597728eaa44150d96c48 but expected c83ce5183b7a9fa525301ab7426d1943808587b2        
To https://huggingface.co/subhasisj/vi-finetuned-squad-qa-minilmv2-8
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/subhasisj/vi-finetuned-squad-qa

OSError: remote: Enforcing permissions...        
remote: Allowed refs: all        
remote: error: cannot lock ref 'refs/heads/main': is at c5355ca2a0f4fe545c53597728eaa44150d96c48 but expected c83ce5183b7a9fa525301ab7426d1943808587b2        
To https://huggingface.co/subhasisj/vi-finetuned-squad-qa-minilmv2-8
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/subhasisj/vi-finetuned-squad-qa-minilmv2-8'


In [23]:
trainer.save_model("./vi-finetuned-squad-qa-minilmv2-8/model")

Saving model checkpoint to ./vi-finetuned-squad-qa-minilmv2-8/model
Configuration saved in ./vi-finetuned-squad-qa-minilmv2-8/model\config.json
Model weights saved in ./vi-finetuned-squad-qa-minilmv2-8/model\pytorch_model.bin
tokenizer config file saved in ./vi-finetuned-squad-qa-minilmv2-8/model\tokenizer_config.json
Special tokens file saved in ./vi-finetuned-squad-qa-minilmv2-8/model\special_tokens_map.json
Saving model checkpoint to ./vi-finetuned-squad-qa-minilmv2-8
Configuration saved in ./vi-finetuned-squad-qa-minilmv2-8\config.json
Model weights saved in ./vi-finetuned-squad-qa-minilmv2-8\pytorch_model.bin
tokenizer config file saved in ./vi-finetuned-squad-qa-minilmv2-8\tokenizer_config.json
Special tokens file saved in ./vi-finetuned-squad-qa-minilmv2-8\special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/subhasisj/vi-finetu

In [ ]:
# !tar -zcvf vi-finetuned-squad-qa-minilmv2-32.tar.gz /kaggle/working/vi-finetuned-squad-qa-minilmv2-32/

In [ ]:
# import os
# os.chdir(r'/kaggle/working')
# from IPython.display import FileLink
 
# FileLink(r'./vi-finetuned-squad-qa-minilmv2-32.tar.gz')

In [ ]:
trainer.push_to_hub("Training Completed.")

In [ ]:
validation_features = squad["validation"]

In [ ]:
# validation_features = squad_valid.map(
#     prepare_validation_features,
#     batched=True,
#     remove_columns=squad_valid.column_names
# )

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
import collections

examples = squad_valid
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(squad_valid, validation_features, raw_predictions.predictions)


In [ ]:
from datasets import load_metric
metric = load_metric("squad")

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in squad_valid]
metric.compute(predictions=formatted_predictions, references=references)